In [1]:
library(dplyr)
library(ggplot2)
library(lemon)
library(gridExtra)
library(gtable)
library(egg)

eval_file = '/labs/shahlab/projects/agataf/data/cohorts/pooled_cohorts/experiments/bmj_review/bootstrap_standard_eval_logit1_fixed.csv'
preds0 <- read.csv(eval_file)

# TODO: with newest version of Stephen's repo, do I still need to make
# those transformations? Can I do them when I run the bootstrap?

# TODO: make this cleaner

# Step 1: transform metrics generated by the bootstrap evaluation 
# to metrics we'd like to plot
preds <- bind_rows(preds0, 
                      preds0 %>%
                        filter(metric=='tce_logit') %>% 
                        mutate(
                             CI_med = CI_med*-1, 
                               CI_lower_new = CI_upper*-1,
                               CI_upper = CI_lower*-1,
                               CI_lower = CI_lower_new,
                               metric='threshold_error',
                                
                        yline=0),
                       preds0 %>% 
                        filter(metric=='sensitivity' | metric=='specificity') %>% 
                        mutate(CI_med = 1-CI_med, 
                               CI_lower_new = 1-CI_upper,
                               CI_upper = 1-CI_lower,
                               CI_lower = CI_lower_new,
                               metric=ifelse(metric=='sensitivity', 'fnr', 
                                              ifelse(metric=='specificity', 'fpr', NA_real_)))

                      )

# TODO: make this switch happen in the bootstrap file.
# Update names to those we'd like to plot
df_to_plot <- preds %>% 
              mutate(group = as.character(group),
                     metric_full_name = recode(metric, 
                                               fnr='False Negative Rate',
                                               fpr='False Positive Rate',
                                     sensitivity = "Sensitivity", 
                                     specificity = "Specificity",
                                     #impl_threshold_log = "Implied\nThreshold", 
                                     threshold_error = "Threshold Calibration Error",
                                     auc = "AUROC",
                                     loss = "Cross Entropy Loss",
                                     ace_logit = "Abs. Calibration Error", 
                                     ace_bin = "Binned ACE",
                                     auprc="AUPRC"), #%>% 
                      threshold = paste("t =", as.character(thresholds*100), "%") %>% 
                                  factor(levels=c("t = 7.5 %", "t = 20 %"))
                     )


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Warning message:
“package ‘gridExtra’ was built under R version 3.6.3”
Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine

Warning message:
“package ‘egg’ was built under R version 3.6.3”Warning message in bind_rows_(x, .id):
“binding factor and character vector, coercing into character vector”Warning message in bind_rows_(x, .id):
“binding character and factor vector, coercing into character vector”

In [2]:
get_overalls <- function(df, met, mod_type) {
    a = df %>% 
    filter(metric==met, 
           model_type==mod_type, 
           group=='overall') %>%
    select(CI_med, CI_lower, CI_upper) %>% 
    round(digits=3)
    return(a)
}

get_max_diff <- function(df, met, mod_type) {
    a = df %>% 
    filter(metric==met, model_type==mod_type) %>% 
    select(CI_med)
    return(round(max(a)-min(a), digits=3))
}

get_overalls_thresh <- function(df, met, mod_type, thresh) {
    a = df %>% 
    filter(metric==met, 
           model_type==mod_type, 
           group=='overall',
          thresholds==thresh) %>%
    select(CI_med, CI_lower, CI_upper) %>% 
    round(digits=3)
    return(a)
}

get_max_diff_thresh <- function(df, met, mod_type, thresh) {
    a = df %>% 
    filter(metric==met, model_type==mod_type, thresholds==thresh) %>% 
    select(CI_med)
    return(round(max(a)-min(a), digits=3))
}

get_group_thresh <- function(df, met, mod_type, thresh, grp) {
    a = df %>% 
    filter(metric==met, 
           model_type==mod_type, 
           #group=='overall',
          thresholds==thresh,
          group==grp) %>%
    select(CI_med, CI_lower, CI_upper) %>% 
    round(digits=3)
    return(a)
}

In [3]:
# AUROC

print("AUROC, UC")
get_overalls(df_to_plot, met='auc', mod_type='UC')

print("AUROC, PCE")
get_overalls(df_to_plot, met='auc', mod_type='PCE')

print("AUROC, rPCE")
get_overalls(df_to_plot, met='auc', mod_type='rPCE')

print("AUROC, EO3")
get_overalls(df_to_plot, met='auc', mod_type='EO3')


# ACE
print("ACE, UC")
get_overalls(df_to_plot, met='ace_logit', mod_type='UC')
print("ACE, PCE")
get_overalls(df_to_plot, met='ace_logit', mod_type='PCE')
print("ACE, rPCE")
get_overalls(df_to_plot, met='ace_logit', mod_type='rPCE')

# TCE

# get_overalls_thresh(df_to_plot, met='tce_logit',
#                     mod_type='UC', thresh=0.075)

# get_overalls_thresh(df_to_plot, met='tce_logit',
#                     mod_type='PCE', thresh=0.075)

# get_overalls_thresh(df_to_plot, met='tce_logit',
#                     mod_type='rPCE', thresh=0.075)
# print("TCE, UC, diff")
# get_max_diff_thresh(df_to_plot, 
#                     met='threshold_error', 
#                     mod_type='UC',
#                    thresh=0.075)
# print("TCE, rPCE, diff")
# get_max_diff_thresh(df_to_plot, 
#                     met='threshold_error', 
#                     mod_type='rPCE',
#                    thresh=0.075)

# TCE - rUC
print("TCE, rPCE, 7.5")
get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='rPCE', thresh=0.075)

print("TCE, UC, 7.5")
get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='UC', thresh=0.075)

                    
print("TCE, rUC, 7.5")
get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='rUC', thresh=0.075)
                

print("TCE, UC, 20")
get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='UC', thresh=0.2)


print("TCE, rUC, 20")
get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='rUC', thresh=0.2)


[1] "AUROC, UC"


CI_med,CI_lower,CI_upper
0.827,0.8,0.853


[1] "AUROC, PCE"


CI_med,CI_lower,CI_upper
0.808,0.779,0.835


[1] "AUROC, rPCE"


...

CI_med,CI_lower,CI_upper
-0.001,-0.007,0.006


[1] "TCE, UC, 20"


CI_med,CI_lower,CI_upper
0.006,-0.013,0.023


[1] "TCE, rUC, 20"


CI_med,CI_lower,CI_upper
0,-0.019,0.016


In [4]:
# TCE - EO
get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='UC', thresh=0.075)

get_overalls_thresh(df_to_plot, met='threshold_error',
                        mod_type='EO3', thresh=0.075)

get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='UC', thresh=0.2)

get_overalls_thresh(df_to_plot, met='threshold_error',
                    mod_type='EO3', thresh=0.2)



CI_med,CI_lower,CI_upper
0.012,0.006,0.019


CI_med,CI_lower,CI_upper
0.033,0.025,0.047


CI_med,CI_lower,CI_upper
0.006,-0.013,0.023


CI_med,CI_lower,CI_upper
-0.236,-0.335,-0.129


In [5]:

get_overalls(df_to_plot, met='ace_logit', mod_type='UC')

CI_med,CI_lower,CI_upper
0.011,0.006,0.023


In [6]:
get_group_thresh(df_to_plot, met='threshold_error',
                    mod_type='UC', thresh=0.2, grp='3') 

get_group_thresh(df_to_plot, met='threshold_error',
                    mod_type='rUC', thresh=0.2, grp='3')

get_overalls_thresh(df_to_plot, met='fnr',
                    mod_type='UC', thresh=0.075)

get_overalls_thresh(df_to_plot, met='fnr',
                    mod_type='rUC', thresh=0.075)

get_overalls_thresh(df_to_plot, met='fpr',
                    mod_type='UC', thresh=0.075)

get_overalls_thresh(df_to_plot, met='fpr',
                    mod_type='rUC', thresh=0.075)


CI_med,CI_lower,CI_upper
0.033,-0.016,0.066


CI_med,CI_lower,CI_upper
-0.071,-0.196,0.008


CI_med,CI_lower,CI_upper
0.179,0.126,0.237


CI_med,CI_lower,CI_upper
0.231,0.171,0.294


CI_med,CI_lower,CI_upper
0.322,0.302,0.344


CI_med,CI_lower,CI_upper
0.265,0.247,0.284
